In [ ]:
# Boilerplate
import sys
import numpy as np
import tensorflow as tf
import matplotlib
from matplotlib.pyplot import figure, imshow, gca, tight_layout, show
from matplotlib import patches
from importlib import reload
import os

if 'roi_pooling' not in sys.modules:
    import backbone
    import classifier
    import data_utils
    import faster_rcnn
    import rpn
    import roi_pooling
    import geometry
    import evaluation
else:
    reload(backbone)
    reload(classifier)
    reload(data_utils)
    reload(faster_rcnn)
    reload(rpn)
    reload(roi_pooling)
    reload(geometry)
    reload(evaluation)

In [ ]:
# List physical devices
is_colab = len(tf.config.list_physical_devices('GPU')) > 0

# Data locations
if is_colab:
    datapath='/content'
    backbone_weights='drive/MyDrive/trained_backbone.ckpt'
    rpn_weights='drive/MyDrive/trained_rpn.ckpt'
    class_weights='drive/MyDrive/trained_classifier.ckpt'
else:
    datapath='tensorflow-great-barrier-reef'
    backbone_weights='trained_backbone.ckpt'
    rpn_weights='trained_rpn.ckpt'
    class_weights='trained_classifier.ckpt'

In [ ]:
# Instantiate the high-level wrapper
frcnn = faster_rcnn.FasterRCNNWrapper(
    input_shape=(720, 1280, 3),
    datapath=datapath,
    backbone_type='ResNet50',
    backbone_weights=backbone_weights if os.path.exists(backbone_weights) else 'finetune',
    rpn_weights=rpn_weights if os.path.exists(rpn_weights) else None,
    classifier_weights=class_weights if os.path.exists(class_weights) else None
)

In [ ]:
training=frcnn.data_loader_full.get_training().__iter__()
x, y_true = training.next()
print(x.shape)
print(y_true)

In [ ]:
features = frcnn.backbone(x)
y_pred = frcnn.rpnwrapper.rpnmodel(features)
print(y_pred.shape)

In [ ]:
reload(evaluation)
metric = evaluation.TopNRegionsF2(20, frcnn.data_loader_full.decode_label)
metric.update_state(y_true, y_pred)

In [ ]:
metric.result()